In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import statsmodels.formula.api as sm
import re
from mpl_toolkits import mplot3d
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

dfs = pd.read_excel("Pew.xlsx", sheet_name=None)

def normalizer(sheet_name):
    #strong_leader_df name means nothing, just a var name based on base case
    #ANY Data Sheet name can be passed in
    strong_leader_df = dfs[sheet_name]
    strong_leader_df.columns = ['Countries', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    strong_leader_df = strong_leader_df.drop(0)
    if(sheet_name == 'Q29c Rule by strong leader'):
        strong_leader_df = strong_leader_df.drop(39)
    strong_leader_df = strong_leader_df.set_index('Countries')
    strong_leader_df = strong_leader_df.drop('1', axis=1)
    strong_leader_df.columns = ['Very good', 'Somewhat good', 'Somewhat bad', 'Very bad', 'DK/Refused', 'Total', 'Good (total)', 'Bad (total)']
    strong_leader_normalized = (strong_leader_df['Good (total)'] - np.average(strong_leader_df['Good (total)']))/(np.std(strong_leader_df['Good (total)']))
    strong_leader_normalized = strong_leader_normalized.drop(['Lebanon', 'Senegal', 'Venezuela', 'Tanzania', 'Nigeria', 'Tunisia', 'Japan', 'Argentina'])
    return strong_leader_normalized

features_ds = pd.DataFrame()
features_ds['rule by strong leader'] = normalizer('Q29c Rule by strong leader') 
features_ds['rule by military'] = normalizer('Q29e Rule by military') 
features_ds['direct democracy'] = normalizer('Q29a direct democracy')
features_ds['rep democracy'] = normalizer('Q29b representative democracy')
features_ds['gov trust'] = normalizer('Q4 Trust in government')
features_ds['dem satisfac'] = normalizer('Q9 Satisfaction with democracy')
features_ds['expert rule'] = normalizer('Q29d Rule by experts')
features_ds.index = features_ds.index.str.lower()
features_ds = features_ds.fillna(0)
features_ds.head(10)

#BREAK

All_Countries = pd.DataFrame()

#countries = ['australia', 'brazil', 'canada', 'chile', 'colombia', 'france', 'germany', 'ghana', 'greece', 'hungary', 'india', 'indoensia', 'israel', 'italy', 'jordan', 'kenya', 'mexico', 'netherlands', 'peru', 'philippines', 'poland', 'russia', 'south africa', 'south korea', 'spain', 'sweden', 'turkey', 'united kingdom', 'united states', 'vietnam']
countries = features_ds.index.str.lower().values
for filename in countries:
    dfs = pd.read_csv(f'Countrys AQI/{filename} aqi.csv')
    if filename == 'ghana' or filename == 'russia':
        dfs = dfs.loc[dfs['date'].str.findall('2020').agg(len) == 1]
    elif filename == 'philippines':    
        dfs = dfs.loc[dfs['date'].str.findall('2018').agg(len) == 1]
    else:
        dfs = dfs.loc[dfs['date'].str.findall('2019').agg(len) == 1]
    dfs = dfs.drop(['date'], axis=1)
    for column in dfs.columns:
        dfs[column] = dfs[column].replace(' ', 0)
    dfs = dfs.astype(float)
    All_Countries[filename] = [max(dfs.apply(np.mean, axis=0))]
All_Countries = All_Countries.transpose()
All_Countries.columns = ['max particulate concentration']
All_Countries.head(10)

#BREAK

pd.set_option("display.max_rows", None)
dfs = pd.read_csv(f'worldcities.csv')
dfs = dfs[dfs['capital']=='primary']
dfs['North'] = (dfs['lat'] > 0).astype(int)
dfs['South'] = (dfs['lat'] < 0).astype(int)
dfs['East'] = (dfs['lng'] > 0).astype(int)
dfs['West'] = (dfs['lng'] < 0).astype(int)
dfs['country'] = dfs['country'].str.lower()
dfs = dfs[['country', 'North', 'South', 'East', 'West']]
dfs.drop_duplicates(inplace=True)
dfs = dfs.set_index('country')
dfs = dfs.rename(index={'korea, south': 'south korea'})
#reintroduce candada since 'ottawa' isn't marked capital
dfs = dfs.append(pd.DataFrame(index=['canada'], data={'North': 1, 'South': 0, 'East': 0, 'West': 1}))
all_features_ds = features_ds.join(dfs)
all_features = all_features_ds.drop(['South', 'West'], axis=1)
all_features.head(10)

#Q11a

accuracy_array = []
for i in range(10):
    train, test = train_test_split(all_features.join(All_Countries), test_size=0.2)
    X_train = train[['rule by strong leader', 'rule by military', 'rep democracy']].to_numpy()
    y_train = train['max particulate concentration']

    X_test = test[['rule by strong leader', 'rule by military', 'rep democracy']].to_numpy()
    y_test = test['max particulate concentration']

    reg = LinearRegression().fit(X_train, y_train)
    accuracy = pd.DataFrame()
    accuracy['true'] = y_test
    accuracy['predicted'] = reg.predict(X_test)
    accuracy_array.append(r2_score(accuracy['true'], accuracy['predicted']))
    
print("just 3 q's", np.average(accuracy_array))

just 3 q's -3.4564098753636343


In [6]:
#Q11c(i), adding rest of questions
accuracy_array = []
for i in range(10):
    train, test = train_test_split(all_features.join(All_Countries), test_size=0.2)
    X_train = train.drop(['max particulate concentration', 'North', 'East'], axis=1).to_numpy()
    y_train = train['max particulate concentration']

    X_test = test.drop(['max particulate concentration', 'North', 'East'], axis=1).to_numpy()
    y_test = test['max particulate concentration']

    better_reg = LinearRegression().fit(X_train, y_train)
    accuracy = pd.DataFrame()
    accuracy['true'] = y_test
    accuracy['predicted'] = better_reg.predict(X_test)
    accuracy_array.append(r2_score(accuracy['true'], accuracy['predicted']))
    
print("adding rest of questions", np.average(accuracy_array))

#Q11c(ii), adding direction too
accuracy_array = []
for i in range(10):
    train, test = train_test_split(all_features.join(All_Countries), test_size=0.2)
    X_train = train.drop(['max particulate concentration'], axis=1).to_numpy()
    y_train = train['max particulate concentration']

    X_test = test.drop(['max particulate concentration'], axis=1).to_numpy()
    y_test = test['max particulate concentration']

    better_reg = LinearRegression().fit(X_train, y_train)
    accuracy = pd.DataFrame()
    accuracy['true'] = y_test
    accuracy['predicted'] = better_reg.predict(X_test)
    accuracy_array.append(r2_score(accuracy['true'], accuracy['predicted']))
    
print("adding direction too", np.average(accuracy_array))

adding rest of questions -3.2089231072999
adding direction too -4.804775631058957


In [7]:
left_leaning = dfs[(dfs['direct democracy'] > 0) & (dfs['rule by military'] < 0) & (dfs['rule by strong leader'] < 0)]
right_leaning = dfs[(dfs['direct democracy'] < 0) & (dfs['rule by military'] > 0) & (dfs['rule by strong leader'] > 0)]
center_leaning = dfs[(dfs.merge(left_leaning.drop_duplicates(), how='left', indicator=True)['_merge'] == 'left_only').array]
center_leaning = dfs[(dfs.merge(right_leaning.drop_duplicates(), how='left', indicator=True)['_merge'] == 'left_only').array]
center_leaning

plt.figure(figsize=(10,8))
plt.title('Average Daily AQI vs. Political Leaning')
barplot_tbl = pd.DataFrame({'political': ['Left', 'Center', 'Right'], 'average aqi': [np.mean(left_leaning['max particulate concentration']),np.mean(center_leaning['max particulate concentration']), np.mean(right_leaning['max particulate concentration'])]})
sns.barplot(x='political', y='average aqi', data=barplot_tbl);
plt.xlabel('Political Label')
plt.ylabel('Average Daily AQI')

KeyError: 'direct democracy'

In [4]:
import plotly.express as px
fig = px.scatter_3d(dfs, x='rule by strong leader', y='direct democracy', z='rule by military',
              color='>50 concentration', color_discrete_sequence=px.colors.qualitative.Pastel,
                title="AQI Concentration indicator in Political Answer Choices")
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['North', 'South', 'East', 'West'] but received: rule by strong leader

In [9]:
dfs = pd.read_excel(f'cities_population.xlsx')

,Name,Type,County,Population (2010),Land area,Incorporated
0,Adelanto,City,San Bernardino,31765,56.01 sq mi (145.1 km2),"December 22, 1970"
1,Agoura Hills,City,Los Angeles,20330,7.79 sq mi (20.2 km2),"December 8, 1982"
2,Alameda,City,Alameda,73812,10.61 sq mi (27.5 km2),"April 19, 1854"
3,Albany,City,Alameda,18539,1.79 sq mi (4.6 km2),"September 22, 1908"
4,Alhambra,City,Los Angeles,83089,7.63 sq mi (19.8 km2),"July 11, 1903"
5,Aliso Viejo,City,Orange,47823,7.47 sq mi (19.3 km2),"July 1, 2001"
6,Alturas,City,Modoc,2827,2.43 sq mi (6.3 km2),"September 16, 1901"
7,Amador City,City,Amador,185,0.31 sq mi (0.80 km2),"June 2, 1915"
8,American Canyon,City,Napa,19454,4.84 sq mi (12.5 km2),"January 1, 1992"
9,Anaheim,City,Orange,336265,49.84 sq mi (129.1 km2),"March 18, 1876"
